In [1]:
from src.utils import *
import numpy as np

/home/steinad/anaconda3/envs/lexx/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
roberta_topic_vals = load("topic_vals_roberta_cola").reshape(1043, 73)
roberta_word_vals = load("word_vals_roberta_cola")
gpt2_topic_vals = load("topic_vals_gpt2_cola").reshape(1043, 73)
gpt2_word_vals = load("word_vals_gpt2_cola")

In [3]:
print(roberta_word_vals[0])

[-0.10056100277738136, 0.3751748564907096, 0.2852067861028693, -0.14280781161959294, 0.4675968735200631, -0.15299237531597737, 0.17787551122729656, 0.0034729123147793223, 0.02383104563404473]


In [4]:
diffs = np.sum(np.abs(roberta_topic_vals - gpt2_topic_vals), axis=1)
biggest_diff = np.argsort(diffs)
print(diffs[biggest_diff[-5:]])

[2.10278409 2.10327668 2.11103148 2.15267332 2.43413353]


In [13]:
config = {"dataset": "cola", "topics": "liwc"}
data_train, data_val = load_data(config)
x = [data_val[i]['sentence'] for i in range(len(data_val))]
x = [xi for xi in x if len(xi.split()) > 1]
topics, topic_names, word2idx = get_topics(config, data_train)

Found cached dataset glue (/home/steinad/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Found cached dataset glue (/home/steinad/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


['ACHIEVE' 'ADJ' 'ADVERB' 'AFFECT' 'AFFILIATION' 'ANGER' 'ANX' 'ARTICLE'
 'ASSENT' 'AUXVERB' 'BIO' 'BODY' 'CAUSE' 'CERTAIN' 'COGPROC' 'COMPARE'
 'CONJ' 'DEATH' 'DIFFER' 'DISCREP' 'DRIVES' 'FAMILY' 'FEEL' 'FEMALE'
 'FILLER' 'FOCUSFUTURE' 'FOCUSPAST' 'FOCUSPRESENT' 'FRIEND' 'FUNCTION'
 'HEALTH' 'HEAR' 'HOME' 'I' 'INFORMAL' 'INGEST' 'INSIGHT' 'INTERROG'
 'IPRON' 'LEISURE' 'MALE' 'MONEY' 'MOTION' 'NEGATE' 'NEGEMO' 'NETSPEAK'
 'NONFLU' 'NUMBER' 'PERCEPT' 'POSEMO' 'POWER' 'PPRON' 'PREP' 'PRONOUN'
 'QUANT' 'RELATIV' 'RELIG' 'REWARD' 'RISK' 'SAD' 'SEE' 'SEXUAL' 'SHEHE'
 'SOCIAL' 'SPACE' 'SWEAR' 'TENTAT' 'THEY' 'TIME' 'VERB' 'WE' 'WORK' 'YOU']
('heed', {'FOCUSPRESENT', 'COGPROC', 'INSIGHT', 'VERB'})


In [17]:
ind = 0
print(x[ind])
topic_values, sorted_names = sort_shap(roberta_topic_vals[ind], topic_names)
for i in range(5):
    print(f"{sorted_names[i]}: {topic_values[i]}")
for i in range(5, 0, -1):
    print(f"{sorted_names[-i]}: {topic_values[-i]}")
print(np.sum(topic_values))

print()
topic_values, sorted_names = sort_shap(gpt2_topic_vals[ind], topic_names)
for i in range(5):
    print(f"{sorted_names[i]}: {topic_values[i]}")
for i in range(5, 0, -1):
    print(f"{sorted_names[-i]}: {topic_values[-i]}")
print(np.sum(topic_values))

The sailors rode the breeze clear of the rocks.
ADJ: -0.030598475063195477
PERCEPT: -0.030598475063195477
COGPROC: -0.030598475063195477
CERTAIN: -0.030598475063195477
SEE: -0.030598475063195477
FUNCTION: 0.08893775561364828
PREP: 0.08893775561364828
HEALTH: 0.2089022911875408
SEXUAL: 0.2089022911875408
BIO: 0.2089022911875408
0.936796795576811

HEALTH: -0.024760359525680516
BIO: -0.024760359525680516
SEXUAL: -0.024760359525680516
ACHIEVE: 0.0
NUMBER: 0.0
COGPROC: 0.02237122416496277
ADJ: 0.02237122416496277
PERCEPT: 0.02237122416496277
FUNCTION: 0.06132630407810212
PREP: 0.06132630407810212
0.19042205810546886


In [15]:
roberta_feat_imp = np.sum(np.abs(roberta_topic_vals), axis=0)
gpt2_feat_imp = np.sum(np.abs(gpt2_topic_vals), axis=0)

topic_values, sorted_names = sort_shap(roberta_feat_imp, topic_names)
for i in range(5):
    print(f"{sorted_names[i]}: {topic_values[i]}")
for i in range(5, 0, -1):
    print(f"{sorted_names[-i]}: {topic_values[-i]}")

print()
topic_values, sorted_names = sort_shap(gpt2_feat_imp, topic_names)
for i in range(5):
    print(f"{sorted_names[i]}: {topic_values[i]}")
for i in range(5, 0, -1):
    print(f"{sorted_names[-i]}: {topic_values[-i]}")

RELIG: 0.0
ASSENT: 0.0
NETSPEAK: 0.0
FILLER: 0.0
NONFLU: 0.015439969404704039
VERB: 45.059353625680785
FUNCTION: 60.41532243189281
SEXUAL: 148.29915543609349
HEALTH: 148.92181427319204
BIO: 151.7978565769454

I: 0.0
NETSPEAK: 0.0
RELIG: 0.0
FILLER: 0.0
ASSENT: 0.0
VERB: 31.993656075250907
FUNCTION: 40.281780652241046
HEALTH: 74.76326663997835
BIO: 74.9713237520887
SEXUAL: 75.0020561495783
